In [44]:
# Importing libraries
import pandas as pd
def recommendation():
    
    # Read csv file into a pandas dataframe
    df = pd.read_csv("books.csv")
    df = df[df.language_code =='eng']
    df['Title'] = df['title'].str.split('(').str[0]
    print("Enter rating w/wo <rating> or random for book suggestion")
    search= input("enter to search ")
    if search=="random":
        df_new= df.sample(replace=True)[['Title', 'authors',"rating"]]
        print("Title: ", df_new[['Title']].to_string(index=False, header=False))
        print("Author: ", df_new[['authors']].to_string(index=False, header=False))
        print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
        return "Enjoy!"
    else:
        if "rating" in search:
            uboundgiven = False
            search=search.split()
            if len(search)>3 or len(search)==1:
                print("Usage: rating lower_value [higher value]")
                return "Please Correct"
            
            rate = float(search[1])
            if len(search)==3:
                uboundgiven = True
                urate = float(search[2])
                
            if not uboundgiven:               
                if rate < 0 or rate > 5:
                    print("Error: Enter rating between 0 and 5")
                    return "Please Correct"
                else:
                    df_rate = df[df['rating'].astype(float) >= rate]
                    df_new= df_rate.sample(replace=True)[['Title', 'authors',"rating"]]
                    print("Title: ", df_new[['Title']].to_string(index=False, header=False))
                    print("Author: ", df_new[['authors']].to_string(index=False, header=False))
                    print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
                    return "Enjoy!"
                
            else:
                if urate < 0 or urate > 5:
                    print("Error: Enter rating between 0 and 5")
                    return "Please Correct"
                elif urate < rate:
                    print("Error: lower value must be lower than higher value.\nUsage: rating lower_value [higher value]")
                    return "Please Correct"
                else:
                    df_upper = df[df['rating'].astype(float) >= rate]
                    df_lower = df[df['rating'].astype(float) <= urate]
                    df_rate = pd.merge(df_upper, df_lower, how='inner', on=['bookID', 'title', 'authors', 'rating', 'language_code', '  num_pages','ratings_count','ratings_count', 'text_reviews_count'])
                    df_new= df_rate.sample(replace=True)[['title', 'authors',"rating"]]
                    print("Title: ", df_new[['title']].to_string(index=False, header=False))
                    print("Author: ", df_new[['authors']].to_string(index=False, header=False))
                    print("Rating: ", df_new[['rating']].to_string(index=False, header=False))
                    return "Enjoy!"
print(recommendation())

Enter rating w/wo <rating> or random for book suggestion
enter to search rating 1.5 1.7
Title:   The Art of Loving by Erich Fromm: A True Story of a Japanese Woman
Author:   Lala Okamoto
Rating:   1.67
Enjoy!


In [37]:
df = pd.read_csv("books.csv")
df = df[df.language_code =='eng']
df['Title'] = df['title'].str.split('(').str[0]

df_upper = df[df['rating'].astype(float) >= 3.0]
df_lower = df[df['rating'].astype(float) <= 4.0]
df_rate = pd.merge(df_upper, df_lower, how='inner', on=['bookID', 'title', 'authors', 'rating', 'language_code', '  num_pages','ratings_count','ratings_count', 'text_reviews_count'])
df_new= df_rate.sample(replace=True)[['title', 'authors',"rating"]]
print(df_new)

                                               title  \
3052  The Road to Jerusalem (The Knight Templar  #1)   

                        authors rating  
3052  Jan Guillou/Anna Paterson      4  
